In [14]:
import os
os.environ["GOOGLE_API_KEY"] = "your key"
google_api_key = os.getenv("GOOGLE_API_KEY")


In [15]:
import os
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Fetch API key securely
google_api_key = os.getenv("GOOGLE_API_KEY")

# Check if API key is found
if not google_api_key:
    raise ValueError("Google API key not found. Please check your .env file.")

# Initialize GoogleGenerativeAI model
llm = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=google_api_key)

# Example: Generate text using the model
response = llm.invoke("What is the capital of France?")
print("Gemini Response:", response)


Gemini Response: Paris


In [16]:
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


In [17]:
import os
from dotenv import load_dotenv
# LangChain core modules
from langchain_google_genai import GoogleGenerativeAI
from langchain.sql_database import SQLDatabase

from langchain_experimental.sql import SQLDatabaseChain

from langchain.vectorstores import FAISS, Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
# MySQL database connectors
import pymysql
import mysql.connector


In [19]:
# Database connection parameters
db_user = "root"
db_password = "12345"
db_host = "localhost"
db_name = "retail_sales_db"

# Create SQLAlchemy engine
db= SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE sales_tb (
	`TransactionID` INTEGER, 
	`Date` DATE, 
	`CustomerID` VARCHAR(10), 
	`Gender` VARCHAR(10), 
	`Age` INTEGER, 
	`ProductCategory` VARCHAR(50), 
	`Quantity` INTEGER, 
	`PriceperUnit` DECIMAL(10, 2), 
	`TotalAmount` DECIMAL(10, 2)
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from sales_tb table:
TransactionID	Date	CustomerID	Gender	Age	ProductCategory	Quantity	PriceperUnit	TotalAmount
1	2023-11-24	CUST001	Male	34	Beauty	3	50.00	150.00
2	2023-02-27	CUST002	Female	26	Clothing	2	500.00	1000.00
3	2023-01-13	CUST003	Male	50	Electronics	1	30.00	30.00
*/


In [20]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm,db)
response = chain.invoke({"question":"How many customers are there"})
response

'SELECT COUNT(DISTINCT `CustomerID`) FROM `sales_tb`'

In [22]:
cleaned_query =response.strip('```sql\n').strip('\n```')
print(cleaned_query)

SELECT COUNT(DISTINCT `CustomerID`) FROM `sales_tb


In [24]:
# result = db.run(cleaned_query)
# print(result)

fixed_query = "SELECT COUNT(DISTINCT `CustomerID`) FROM `sales_tb`"
result = db.run(fixed_query)
print(result)


[(29,)]


In [27]:
chain = create_sql_query_chain(llm,db)
def execute_query(question):
    try:
        response = chain.invoke({"question":question})
        print(response)
        print('#########################################')
        cleaned_query =response.strip('```sql\n').strip('\n```')
        print(cleaned_query)
        print('#########################################')
        result = db.run(cleaned_query)
        print('#########################################')
        print(result)
    except ProgrammingError as e:
        print(''f"an error occured:{e}")
    
        
        

In [28]:
q1 = "how many unique customers are therefor each product category"
execute_query(q1)

```sql
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS UniqueCustomerCount
FROM `sales_tb`
GROUP BY `ProductCategory`;
```
#########################################
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS UniqueCustomerCount
FROM `sales_tb`
GROUP BY `ProductCategory`;
#########################################
#########################################
[('Beauty', 8), ('Clothing', 13), ('Electronics', 8)]


In [29]:
q2 = 'calculate total sales amount per product category'
execute_query(q2)

```sql
SELECT `ProductCategory`, SUM(`TotalAmount`) AS TotalSalesAmount FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
```
#########################################
SELECT `ProductCategory`, SUM(`TotalAmount`) AS TotalSalesAmount FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
#########################################
#########################################
[('Beauty', Decimal('1455.00')), ('Clothing', Decimal('5040.00')), ('Electronics', Decimal('5310.00'))]


In [30]:
q3 = "calculate the average age of customers grouped by gender"
execute_query(q3)


```sql
SELECT `Gender`, AVG(`Age`) AS average_age FROM `sales_tb` GROUP BY `Gender` LIMIT 5;
```
#########################################
SELECT `Gender`, AVG(`Age`) AS average_age FROM `sales_tb` GROUP BY `Gender` LIMIT 5;
#########################################
#########################################
[('Male', Decimal('35.2143')), ('Female', Decimal('43.3333'))]


In [31]:
q4 = "identify the top spending customers based on thier total amount spent"
execute_query(q3)


```sql
SELECT `Gender`, AVG(`Age`) AS average_age FROM `sales_tb` GROUP BY `Gender` LIMIT 5;
```
#########################################
SELECT `Gender`, AVG(`Age`) AS average_age FROM `sales_tb` GROUP BY `Gender` LIMIT 5;
#########################################
#########################################
[('Male', Decimal('35.2143')), ('Female', Decimal('43.3333'))]


In [1]:
# import os
# import streamlit as st
# from langchain.chains import create_sql_query_chain
# from langchain_google_genai import GoogleGenerativeAI
# from sqlalchemy import create_engine
# from sqlalchemy.exc import ProgrammingError
# from langchain_community.utilities import SQLDatabase
# from dotenv import load_dotenv
# load_dotenv() 

# # Database connection parameters
# db_user = "root"
# db_password = "12345"
# db_host = "localhost"
# db_name = "retail_sales_db"

# # Create SQLAlchemy engine
# engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

# # Initialize SQLDatabase
# db = SQLDatabase(engine, sample_rows_in_table_info=3)

# # Initialize LLM
# llm = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.environ["GOOGLE_API_KEY"])

# # Create SQL query chain
# chain = create_sql_query_chain(llm, db)

# def execute_query(question):
#     try:
#         # Generate SQL query from question
#         response = chain.invoke({"question": question})

#         # Execute the query
#         result = db.run(response)
                
#         # Return the query and the result
#         return response, result
#     except ProgrammingError as e:
#         st.error(f"An error occurred: {e}")
#         return None, None

# # Streamlit interface
# st.title("Question Answering App")

# # Input from user
# question = st.text_input("Enter your question:")

# if st.button("Execute"):
#     if question:
#         cleaned_query, query_result = execute_query(question)
        
#         if cleaned_query and query_result is not None:
#             st.write("Generated SQL Query:")
#             st.code(response, language="sql")
#             st.write("Query Result:")
#             st.write(query_result)
#         else:
#             st.write("No result returned due to an error.")
#     else:
#         st.write("Please enter a question.")
        

# Using llama3.2 locally 

In [3]:
import ollama
from langchain_community.llms import Ollama

# Initialize LLaMA 3.2 model locally
llm = Ollama(model="llama3.2")  # No API key required

# Example: Generate text using the model
response = llm.invoke("What is the capital of France?")
print("LLaMA 3.2 Response:", response)


LLaMA 3.2 Response: The capital of France is Paris.


In [8]:
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [9]:
# Import Ollama for locally running LLaMA 3.2
import ollama
from langchain_community.llms import Ollama  # Local LLaMA 3.2 Model

# Database-related imports
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

# Vector database imports (if needed for embeddings)
from langchain.vectorstores import FAISS, Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# MySQL database connectors
import pymysql
import mysql.connector
import os


In [10]:
# Database connection parameters
db_user = "root"
db_password = "12345"
db_host = "localhost"
db_name = "retail_sales_db"

# Create SQLAlchemy engine
db= SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE sales_tb (
	`TransactionID` INTEGER, 
	`Date` DATE, 
	`CustomerID` VARCHAR(10), 
	`Gender` VARCHAR(10), 
	`Age` INTEGER, 
	`ProductCategory` VARCHAR(50), 
	`Quantity` INTEGER, 
	`PriceperUnit` DECIMAL(10, 2), 
	`TotalAmount` DECIMAL(10, 2)
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from sales_tb table:
TransactionID	Date	CustomerID	Gender	Age	ProductCategory	Quantity	PriceperUnit	TotalAmount
1	2023-11-24	CUST001	Male	34	Beauty	3	50.00	150.00
2	2023-02-27	CUST002	Female	26	Clothing	2	500.00	1000.00
3	2023-01-13	CUST003	Male	50	Electronics	1	30.00	30.00
*/


In [11]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm,db)
response = chain.invoke({"question":"How many customers are there"})
response

'Question: How many customers are there\n\nSQLQuery:\n```sql\nSELECT COUNT(DISTINCT `CustomerID`) FROM sales_tb;\n```'

In [12]:
cleaned_query =response.strip('```sql\n').strip('\n```')
print(cleaned_query)

Question: How many customers are there

SQLQuery:
```sql
SELECT COUNT(DISTINCT `CustomerID`) FROM sales_tb;


In [13]:
fixed_query = "SELECT COUNT(DISTINCT `CustomerID`) FROM `sales_tb`"
result = db.run(fixed_query)
print(result)


[(29,)]


In [15]:
chain = create_sql_query_chain(llm,db)
def execute_query(question):
    try:
        response = chain.invoke({"question":question})
        print(response)
        print('#########################################')
        cleaned_query =response.strip('```sql\n').strip('\n```')
        print(cleaned_query)
        print('#########################################')
        result = db.run(cleaned_query)
        print('#########################################')
        print(result)
    except ProgrammingError as e:
        print(''f"an error occured:{e}")  

In [17]:
from pymysql.err import ProgrammingError

q1 = "how many unique customers are therefor each product category"
execute_query(q1)

Question: how many unique customers are there for each product category
SQLQuery: 
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS UniqueCustomers
FROM sales_tb
GROUP BY `ProductCategory`
ORDER BY COUNT(`CustomerID`) DESC
LIMIT 5;
#########################################
Question: how many unique customers are there for each product category
SQLQuery: 
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS UniqueCustomers
FROM sales_tb
GROUP BY `ProductCategory`
ORDER BY COUNT(`CustomerID`) DESC
LIMIT 5;
#########################################


ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Question: how many unique customers are there for each product category\nSQLQuery' at line 1")
[SQL: Question: how many unique customers are there for each product category
SQLQuery: 
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS UniqueCustomers
FROM sales_tb
GROUP BY `ProductCategory`
ORDER BY COUNT(`CustomerID`) DESC
LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [18]:
q2 = 'calculate total sales amount per product category'
execute_query(q2)

Question: calculate total sales amount per product category
SQLQuery:
```sql
SELECT `ProductCategory`, SUM(`TotalAmount`) as TotalSales
FROM sales_tb
GROUP BY `ProductCategory`
ORDER BY SUM(`TotalAmount`) DESC
LIMIT 5;
```
#########################################
Question: calculate total sales amount per product category
SQLQuery:
```sql
SELECT `ProductCategory`, SUM(`TotalAmount`) as TotalSales
FROM sales_tb
GROUP BY `ProductCategory`
ORDER BY SUM(`TotalAmount`) DESC
LIMIT 5;
#########################################


ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Question: calculate total sales amount per product category\nSQLQuery:\n```sql\nSEL' at line 1")
[SQL: Question: calculate total sales amount per product category
SQLQuery:
```sql
SELECT `ProductCategory`, SUM(`TotalAmount`) as TotalSales
FROM sales_tb
GROUP BY `ProductCategory`
ORDER BY SUM(`TotalAmount`) DESC
LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/f405)